In [107]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [114]:
model = load_model("model_dl.h5")
class_labels = ["Full Stop", "Give Way", "Give Way for oncoming traffic", "Priority Over Oncoming Traffic", "Priority Road"]

In [115]:
def get_shape_name(approx, contour):
    vertices = len(approx)
    
    if vertices == 3:
        return "Triangle"
    elif vertices == 4:
        x, y, w, h = cv2.boundingRect(approx)
        aspect_ratio = float(w) / h
        if 0.8 < aspect_ratio < 1.2:
            return "Rhombus"
        else:
            return "Rectangle"
    elif vertices == 6:
        return "Hexagon"
    elif vertices > 6:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            return "Other"
        circularity = 4 * np.pi * (area / (perimeter ** 2))
        if circularity > 0.75:
            return "Circle"
        else:
            return "Other"
    else:
        return "Other"


In [116]:
def detect_shapes(img):
    shapes = []
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        if cv2.contourArea(cnt) < 800:  # adjust threshold as needed
            continue

        approx = cv2.approxPolyDP(cnt, 0.04 * cv2.arcLength(cnt, True), True)

        if len(approx) == 3:
            shape = "triangle"
        elif len(approx) == 4:
            shape = "rectangle"
        elif len(approx) > 6:
            shape = "circle"
        else:
            shape = "unidentified"

        x, y, w, h = cv2.boundingRect(approx)
        roi = img[y:y+h, x:x+w]
        shapes.append((roi, shape))

    return shapes


In [117]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

def detect_shapes(img):
    shapes = []
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area (largest to smallest)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 1000:
            continue  # filter out small noise

        # Check solidity to remove hollow shapes
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        if hull_area == 0 or area / hull_area < 0.9:
            continue

        approx = cv2.approxPolyDP(cnt, 0.015 * cv2.arcLength(cnt, True), True)
        shape = "unidentified"

        if len(approx) == 3:
            shape = "reversed triangle"
        elif len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            if 0.85 < aspect_ratio < 1.15:
                shape = "rhombus"
            else:
                shape = "rectangle"
        elif 5 <= len(approx) <= 7:
            shape = "hexagon"
        elif len(approx) > 7:
            shape = "circle"

        x, y, w, h = cv2.boundingRect(approx)
        roi = img[y:y+h, x:x+w]
        shapes.append((roi, shape))
        break  # only return the biggest likely sign

    return shapes


In [118]:
def process_image(image_path):
    image = cv2.imread(image_path)

    if image is None:
        return "Image not found or unreadable."

    signs = detect_shapes(image)

    if not signs:
        return "No valid traffic signs detected."

    shape_to_label = {
        "circle": "Give Way to Oncoming Traffic",
        "rectangle": "Priority Over Oncoming Traffic",
        "rhombus": "Priority Road",
        "hexagon": "Stop",
        "reversed triangle": "Give Way"
    }

    predictions = []
    for i, (roi, shape) in enumerate(signs):
        label = shape_to_label.get(shape, "Unknown")
        predictions.append((label, shape))
        print(f"[{i+1}] Detected shape: {shape} -> Predicted sign: {label}")

    return predictions


In [119]:
# ------------------ Example Usage -------------------

image_path = "s_picture8.png"  # Replace with your image path
predictions = process_image(image_path)
print("Predictions:", predictions)


[1] Detected shape: circle -> Predicted sign: Give Way to Oncoming Traffic
Predictions: [('Give Way to Oncoming Traffic', 'circle')]
